In [37]:
import pandas as pd

In [ ]:
%pip install openpyxl

In [12]:
%run 0-nettoyage-bdd-v2.ipynb

In [47]:
base=nettoyer_lic_comm("https://www.data.gouv.fr/fr/datasets/r/9b002e32-0ec9-40fc-9292-cddeae4416c6","https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.xlsx")

C:\Users\user\AppData\Local\Temp\ipykernel_2612\775948522.py:5: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  lic_comm = pd.read_csv(chemin_acces_lic_comm, encoding="utf-8", sep=";")


In [48]:
base

,Département,Code,Fédération,Région,categorie,nb_licencies,tranche_age,sexe,Nom de la région,Code département,Nom du département,Population municipale,Population totale
0,01,101,FF d'Athlétisme,Auvergne-Rhône-Alpes,F - 1 à 4 ans,4,1-4,F,Auvergne-Rhône-Alpes,01,Ain,647634,663720
1,01,102,FF d'Aviron,Auvergne-Rhône-Alpes,F - 1 à 4 ans,0,1-4,F,Auvergne-Rhône-Alpes,01,Ain,647634,663720
2,01,103,FF de Badminton,Auvergne-Rhône-Alpes,F - 1 à 4 ans,0,1-4,F,Auvergne-Rhône-Alpes,01,Ain,647634,663720
3,01,105,FF de Basketball,Auvergne-Rhône-Alpes,F - 1 à 4 ans,19,1-4,F,Auvergne-Rhône-Alpes,01,Ain,647634,663720
4,01,106,FF de Boxe,Auvergne-Rhône-Alpes,F - 1 à 4 ans,0,1-4,F,Auvergne-Rhône-Alpes,01,Ain,647634,663720
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313373,974,601,FF du Sport Universitaire,DROM,H - 80 à 99 ans,0,60-99,H,La Réunion,974,La Réunion,855961,865507
313374,974,602,F Sportive Educative de l'Enseignement Catholi...,DROM,H - 80 à 99 ans,0,60-99,H,La Réunion,974,La Réunion,855961,865507
313375,974,603,Union Nationale des Clubs Universitaires,DROM,H - 80 à 99 ans,0,60-99,H,La Réunion,974,La Réunion,855961,865507
313376,974,604,Union Nationale du Sport Scolaire (UNSS),DROM,H - 80 à 99 ans,0,60-99,H,La Réunion,974,La Réunion,855961,865507


In [ ]:
#### taux de licenciés par régions
